In [3]:
import pandas as pd

df = pd.read_csv("crop_yield_50000.csv")
print(df.head())


  Region Soil_Type    Crop  Rainfall_mm  Temperature_Celsius  Fertilizer_Used  \
0  North      Loam   Maize   269.628359            33.568049             True   
1  South      Silt  Barley   277.096946            31.503174            False   
2  South    Chalky   Maize   872.538244            34.665578            False   
3  North      Loam   Wheat   475.905625            28.462172            False   
4  North      Clay    Rice   381.067981            19.256107             True   

   Irrigation_Used Weather_Condition  Days_to_Harvest  Yield_tons_per_hectare  
0             True             Sunny       100.778743                5.015345  
1             True             Rainy        90.869031                2.850527  
2            False             Sunny        84.610447                5.698175  
3             True             Sunny        97.719303                3.840846  
4            False            Cloudy        68.468178                3.166763  


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

X = df.drop("Crop", axis=1)
y = df["Crop"]

le = LabelEncoder()
y = le.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.select_dtypes(include=['float64','int64']))

# Replace scaled values back
X[X.select_dtypes(include=['float64','int64']).columns] = X_scaled

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

X = df.drop("Crop", axis=1)
y = df["Crop"]

le = LabelEncoder()
y = le.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.select_dtypes(include=['float64','int64']))

# Replace scaled values back
X[X.select_dtypes(include=['float64','int64']).columns] = X_scaled

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Encode categorical features using pd.get_dummies
X_train_enc = pd.get_dummies(X_train)
X_test_enc = pd.get_dummies(X_test)

# Align columns in case some categories are missing in test/train
X_train_enc, X_test_enc = X_train_enc.align(X_test_enc, join='left', axis=1, fill_value=0)

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train_enc, y_train)

y_pred = model.predict(X_test_enc)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 0.3865
              precision    recall  f1-score   support

      Barley       0.40      0.43      0.41      1632
      Cotton       0.42      0.40      0.41      1785
       Maize       0.39      0.43      0.41      1648
        Rice       0.43      0.43      0.43      1656
     Soybean       0.42      0.42      0.42      1641
       Wheat       0.22      0.19      0.21      1638

    accuracy                           0.39     10000
   macro avg       0.38      0.39      0.38     10000
weighted avg       0.38      0.39      0.38     10000



In [8]:
# Prepare new sample as a DataFrame with the same columns as X_train
new_sample_dict = {
	'Region': ['North'],  # Example value, update as needed
	'Soil_Type': ['Loamy'],
	'Rainfall_mm': [1200],
	'Temperature_Celsius': [28],
	'Fertilizer_Used': [True],  # Example value, update as needed
	'Irrigation_Used': [True],  # Example value, update as needed
	'Weather_Condition': ['Kharif'],  # This should match your training categories, update as needed
	'Days_to_Harvest': [120],
	'Yield_tons_per_hectare': [3.5]
}
new_sample_df = pd.DataFrame(new_sample_dict)

# Scale numerical columns
num_cols = X.select_dtypes(include=['float64','int64']).columns
new_sample_df[num_cols] = scaler.transform(new_sample_df[num_cols])

# Encode categorical features using pd.get_dummies
new_sample_enc = pd.get_dummies(new_sample_df)

# Align columns with training data
new_sample_enc = new_sample_enc.reindex(columns=X_train_enc.columns, fill_value=0)

# Predict
prediction = model.predict(new_sample_enc)
print("Predicted Crop:", le.inverse_transform(prediction)[0])


Predicted Crop: Wheat
